In [12]:
import numpy as np
import serial
import time
import scipy as sc
import matplotlib.pyplot as plt
import serial

import time


In [66]:
ser

Serial<id=0x7f2e3a013b20, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)

In [13]:
ser=serial.Serial('/dev/ttyUSB1')
Baudrate=115200
ser.baudrate=Baudrate
az_min=0
az_max=360
el_min=0
el_max=90

In [14]:
def SendPacket(packet):
    ser.write(packet)
    ser.flush()
def ReceivePacket():
    return ser.read(12)

def ReceivePacket_az_el():
    received_vals=ReceivePacket()
    az = (
    (received_vals[1] * 100)
    + (received_vals[2] * 10)
    + received_vals[3]
    + (received_vals[4] / 10.0)-az_max+az_min

    )
    el = (
    (received_vals[6] * 100)
    + (received_vals[7] * 10)
    + received_vals[8]
    + (received_vals[9] / 10.0)
    -360.0
    )
    ph = received_vals[5]
    pv = received_vals[10]
    
    print("Azimuth:   " + str(az))
    print("Elevation: " + str(el))
    print("PH: " + str(ph))
    print("PV: " + str(pv) + "\n")
    return az,el

In [15]:

def StatusCmd():
    cmd = [0x57, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x1f, 0x20]
    packet = bytes(cmd)
    SendPacket(packet)
    time.sleep(1)
    az,el=ReceivePacket_az_el()
    

def Stop():
    cmd=[0x57,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x0F,0x20]
    packet=bytes(cmd)
    SendPacket(packet)
    time.sleep(1)
    az,el=ReceivePacket_az_el()

def Restart():
    cmd=[0x57,0xef,0xbe,0xad,0xde,0x00,0x00,0x00,0x00,0x00,0x00,0xee,0x20]
    packet=bytes(cmd)
    SendPacket(packet)
    print('Restarting in')
    for i in range(5, 0, -1):
        print(i)
        time.sleep(1)  # Wait for 1 second
    print("Restarting")
    
def Point(az,el,pulse_per_deg=10):
    pulse=pulse_per_deg
    az=az-az_min
    el=el
    az_pulses=str(int((az+az_max)*pulse))
    el_pulses=str(int((el+360)*pulse))
    
    cmd = [
        0x57,  # Start byte
        0x30 + int(az_pulses[0]),  # Convert each digit to ASCII hex
        0x30 + int(az_pulses[1]),
        0x30 + int(az_pulses[2]),
        0x30 + int(az_pulses[3]),
        0x0A,  # Pulse per degree for azimuth (newline or separator)
        0x30 + int(el_pulses[0]),  # Convert each digit to ASCII hex
        0x30 + int(el_pulses[1]),
        0x30 + int(el_pulses[2]),
        0x30 + int(el_pulses[3]),
        0x0A,  # Pulse per degree for elevation (newline or separator)
        0xF2,  # Arbitrary end or control byte
        0x20   # Space byte
    ]
    packet=bytes(cmd)
    SendPacket(packet)

In [25]:
Point(0,0)

In [16]:
Restart()

Restarting in
5
4
3
2
1
Restarting


In [24]:
StatusCmd()

Azimuth:   50.0
Elevation: 0.0
PH: 10
PV: 10



In [95]:
Stop()

Azimuth:   -180.0
Elevation: 0.0
PH: 10
PV: 10



In [67]:
M=Motor('/dev/ttyUSB0',115200,0,360,-5,90)


TypeError: Can't instantiate abstract class Motor without an implementation for abstract methods 'point', 'status'

In [64]:
import serial

from abc import ABC, abstractmethod
from time import sleep
from math import cos, acos, pi, sqrt, floor


class Motor(ABC):
    """Abstract Class for All Motors Types

    Attributes
    ----------
    port : str
        Serial Port Identifier String for Communicating with the Motor
    baudrate : int
        Baudrate for serial connection
    az_limits : (float, float)
        Tuple of Lower and Upper Azimuth Limits
    el_limits : (float, float)
        Tuple of Lower and Upper Elevation Limits
    serial : serial.Serial
        Serial Object for Communicating with the Motor

    See Also
    --------
    <https://pyserial.readthedocs.io/en/latest/pyserial_api.html>
    """

    def __init__(self, port, baudrate, az_limits, el_limits):
        """Constructor for the Abstract Motor Class

        Parameters
        ----------
        port : str
            Serial Port Identifier String for Communicating with the Motor
        az_limits : (float, float)
            Tuple of Lower and Upper Azimuth Limits
        el_limits : (float, float)
            Tuple of Lower and Upper Elevation Limits
        """
        self.port = port
        self.baudrate = baudrate
        self.az_limits = az_limits
        self.el_limits = el_limits
        self.serial = None

    @abstractmethod
    def point(self, az, el):
        """Abstract Method Prototype for Pointing a Motor at an AzEl Coordinate

        Parameters
        ----------
        az : float
            Azimuth Coordinate Value to Point At
        el : float
            Elevation Coordinate Value to Point At

        Returns
        -------
        (float, float)
            Azimuth and Elevation Coordinate as a Tuple of Floats
        """
        pass

    @abstractmethod
    def status(self):
        """Abstract Method Prototype for Getting a Motor's Current AzEl Position

        Returns
        -------
        (float, float)
            Azimuth and Elevation Coordinate as a Tuple of Floats
        """
        pass

    def __del__(self):
        """Override of Motor Delete Method to Close Serial Port if Necessary

        Returns
        -------
        None
        """
        if self.serial is not None and self.serial.is_open:
            self.serial.close()

In [78]:
Testing=Rot2Motor('/dev/ttyUSB0',115200,[0,360],[0,90],10)

In [10]:
Testing.point(0,0)

NameError: name 'Testing' is not defined

In [117]:
Testing.status()

(-20.0, 0.0)

In [118]:
class Rot2Motor(Motor):
    """Class for Controlling any ROT2 Protocol-Supporting Motor (e.g. SPID Motors)

    See Also
    --------
    <http://ryeng.name/blog/3>
    <https://github.com/jaidenfe/rot2proG/blob/master/rot2proG.py>
    <https://www.haystack.mit.edu/edu/undergrad/srt/pdf%20files/MD-01%20en.pdf>
    """

    VALID_PULSES_PER_DEGREE = (10, 2, 4)

    def __init__(
        self,
        port,
        baudrate,
        az_limits,
        el_limits,
        pulses_per_degree=10,
        test_pulses_per_degree=True,
    ):
        """Initializer for Rot2Motor

        Parameters
        ----------
        port : str
            Serial Port Identifier String for Communicating with the Motor
        baudrate : int
            Baudrate for serial connection
        az_limits : (float, float)
            Tuple of Lower and Upper Azimuth Limits
        el_limits : (float, float)
            Tuple of Lower and Upper Elevation Limits
        pulses_per_degree : int
            Number of Motor Pulses per Degree of Movement
        test_pulses_per_degree : bool
            Whether to Run A Call to Ask the Motor What its True Pulses per Degree Is (By Calling status)
        """
        Motor.__init__(self, port, baudrate, az_limits, el_limits)
        self.serial = serial.Serial(
            port=self.port,
            baudrate=baudrate,
            bytesize=serial.EIGHTBITS,
            parity="N",
            stopbits=serial.STOPBITS_ONE,
            timeout=None,
        )
        if pulses_per_degree in Rot2Motor.VALID_PULSES_PER_DEGREE:
            self.pulses_per_degree = pulses_per_degree
        else:
            raise ValueError("Invalid Pulse Per Degree Value")
        if test_pulses_per_degree:
            self.status()

    def send_rot2_pkt(self, cmd, az=None, el=None):
        """Builds and Sends a ROT2 Command Packet over Serial

        Parameters
        ----------
        cmd : int
            ROT2 Motor Command Value (0x2F -> Set, 0x1F -> Get, 0x0F -> Stop)
        az : float
            Azimuth Coordinate to Point At (If Applicable)
        el : float
            Elevation Coordinate to Point At (If Applicable)

        Notes
        -----
        All send_rot2_pkt calls should be followed with a receive_rot2_pkt

        Returns
        -------
        None
        """
        if az is not None and el is not None:
            azimuth = int(
                self.pulses_per_degree * (az + 360.0)
            )  # Formatted Az Pulse Value
            elevation = int(
                self.pulses_per_degree * (el + 360.0)
            )  # Formatted El Pulse Value
        else:
            azimuth = 0
            elevation = 0

        azimuth_ticks = (
            self.pulses_per_degree
        )  # Documentation for Rot2 Says This Is Ignored
        elevation_ticks = (
            self.pulses_per_degree
        )  # Documentation for Rot2 Says This Is Ignored

        cmd_string = "W%04d%c%04d%c%c " % (
            azimuth,
            azimuth_ticks,
            elevation,
            elevation_ticks,
            cmd,
        )
        cmd_bytes = cmd_string.encode("ascii")
        # print("Packet of Size " + str(len(cmd_bytes)))
        # print([hex(val) for val in cmd_bytes])
        self.serial.write(cmd_bytes)

    def receive_rot2_pkt(self):
        """Receives and Parsers an ROT2 Status Packet

        Returns
        -------
        (float, float)
            Azimuth and Elevation Coordinate as a Tuple of Floats
        """
        received_vals = self.serial.read(12)
        az = (
            (received_vals[1] * 100)
            + (received_vals[2] * 10)
            + received_vals[3]
            + (received_vals[4] / 10.0)
            - 360.0
        )
        el = (
            (received_vals[6] * 100)
            + (received_vals[7] * 10)
            + received_vals[8]
            + (received_vals[9] / 10.0)
            - 360.0
        )
        az_pulse_per_deg = received_vals[5]
        el_pulse_per_deg = received_vals[10]
        assert az_pulse_per_deg == el_pulse_per_deg  # Consistency Check
        if az_pulse_per_deg != self.pulses_per_degree:
            print(
                "Motor Pulses Per Degree Incorrect, Changing Value to "
                + str(az_pulse_per_deg)
            )
            self.pulses_per_degree = az_pulse_per_deg
        return az, el

    def point(self, az, el):
        """Point ROT2 Motor at AzEl Coordinate

        Parameters
        ----------
        az : float
            Azimuth Coordinate to Point At
        el : float
            Elevation Coordinate to Point At

        Returns
        -------
        None
        """
        cmd = 0x2F  # Rot2 Set Command
        az_relative = az - self.az_limits[0]
        el_relative = el - self.el_limits[0]
        self.send_rot2_pkt(cmd, az=az_relative, el=el_relative)

    def status(self):
        """Requests the Current Location of the ROT2 Motor

        Returns
        -------
        (float, float)
            Current Azimuth and Elevation Coordinate as a Tuple of Floats
        """
        cmd = 0x1F  # Rot2 Status Command
        self.send_rot2_pkt(cmd)
        az_relative, el_relative = self.receive_rot2_pkt()
        return az_relative + self.az_limits[0], el_relative + self.el_limits[0]

    def stop(self):
        """Stops the ROT2 Motor at its Current Location

        Returns
        -------
        None
        """
        cmd = 0x0F  # Rot2 Stop Command
        self.send_rot2_pkt(cmd)
        # az_relative, el_relative = self.receive_rot2_pkt()
        # return (az_relative + self.az_limits[0], el_relative + self.el_limits[0])